In [120]:
import pandas as pd
import numpy as np
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy import sparse
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 


In [80]:
news_data = pd.read_csv("News_Final.csv")

In [81]:
news_data = news_data[['Title','Headline','Topic','Facebook', 'GooglePlus', 'LinkedIn']].copy()
news_data.head()

,Title,Headline,Topic,Facebook,GooglePlus,LinkedIn
0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,obama,-1,-1,-1
1,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",economy,-1,-1,-1
2,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",economy,-1,-1,-1
3,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,economy,-1,-1,-1
4,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,economy,-1,-1,-1


In [82]:
news_data['popularity'] = ((news_data.Facebook) + (news_data.GooglePlus) + (news_data.LinkedIn)).astype(int)



In [83]:
news_data = news_data[news_data.popularity >= 0]

news_data = news_data[['Title','Headline', 'Topic','popularity']]
news_data.head()

,Title,Headline,Topic,popularity
6,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",palestine,0
56,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0
111,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0
726,Microsoft offering £100 discount on original B...,Just ahead of the market rollout of Microsoft'...,microsoft,0
727,Economy to improve though no change in last 6 ...,"&quot;In the coming six months, there seems to...",economy,0


In [84]:
news_data['Title'] = news_data['Title'].map(lambda x: x.rstrip('...'))

popularity = news_data[['popularity']]

In [85]:
min_max_scalar = preprocessing.MinMaxScaler()

x_scaled = min_max_scalar.fit_transform(popularity)

df_normalized = pd.DataFrame(x_scaled)

C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [86]:
se = pd.Series(df_normalized[0])

news_data['popular'] = se.values
news_data.drop(['popularity'] ,axis = 1)

,Title,Headline,Topic,popular
6,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",palestine,0.000000
56,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0.000000
111,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0.000000
726,Microsoft offering £100 discount on original B...,Just ahead of the market rollout of Microsoft'...,microsoft,0.000000
727,Economy to improve though no change in last 6 ...,"&quot;In the coming six months, there seems to...",economy,0.000000
728,Economy to improve though no change in last 6 ...,"New Delhi, Nov 8 (IANS) Majority of Indian bus...",economy,0.000000
729,Microsoft scales back free online storage plans,NEW YORK — Microsoft is getting stingy with on...,microsoft,0.000000
730,IDC predicts the emergence of ‘DX economy’,KUCHING: The global digital economy is about t...,economy,0.000020
731,Economic Preview: Whew! U.S. economy hasn’t go...,Seems the economy is doing fine after all. The...,economy,0.000589
732,Microsoft’s OneDrive debacle shows its cloud c...,When Microsoft announced earlier this week tha...,microsoft,0.000142


In [87]:
list = []

for i, row in news_data.iterrows():
    if(row['popular'] == 0):
        list.append(0)
    elif(row['popular'] > 0):
        list.append(1)
        
popular = pd.Series(list)
news_data['popularity'] = popular.values
news_data

,Title,Headline,Topic,popularity,popular
6,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",palestine,0,0.000000
56,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0,0.000000
111,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0,0.000000
726,Microsoft offering £100 discount on original B...,Just ahead of the market rollout of Microsoft'...,microsoft,0,0.000000
727,Economy to improve though no change in last 6 ...,"&quot;In the coming six months, there seems to...",economy,0,0.000000
728,Economy to improve though no change in last 6 ...,"New Delhi, Nov 8 (IANS) Majority of Indian bus...",economy,0,0.000000
729,Microsoft scales back free online storage plans,NEW YORK — Microsoft is getting stingy with on...,microsoft,0,0.000000
730,IDC predicts the emergence of ‘DX economy’,KUCHING: The global digital economy is about t...,economy,1,0.000020
731,Economic Preview: Whew! U.S. economy hasn’t go...,Seems the economy is doing fine after all. The...,economy,1,0.000589
732,Microsoft’s OneDrive debacle shows its cloud c...,When Microsoft announced earlier this week tha...,microsoft,1,0.000142


In [88]:
news_data = news_data.drop(['popular'], axis = 1)

news_data

,Title,Headline,Topic,popularity
6,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",palestine,0
56,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0
111,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",palestine,0
726,Microsoft offering £100 discount on original B...,Just ahead of the market rollout of Microsoft'...,microsoft,0
727,Economy to improve though no change in last 6 ...,"&quot;In the coming six months, there seems to...",economy,0
728,Economy to improve though no change in last 6 ...,"New Delhi, Nov 8 (IANS) Majority of Indian bus...",economy,0
729,Microsoft scales back free online storage plans,NEW YORK — Microsoft is getting stingy with on...,microsoft,0
730,IDC predicts the emergence of ‘DX economy’,KUCHING: The global digital economy is about t...,economy,1
731,Economic Preview: Whew! U.S. economy hasn’t go...,Seems the economy is doing fine after all. The...,economy,1
732,Microsoft’s OneDrive debacle shows its cloud c...,When Microsoft announced earlier this week tha...,microsoft,1


In [89]:
cvec = CountVectorizer(stop_words = 'english', min_df=1)

In [90]:
X = news_data.Title
y = news_data.popularity

X_train,X_test,y_train,y_test = train_test_split(X , y, test_size = 0.20, shuffle = False)

In [91]:
train_title = cvec.fit_transform(X_train)
train_title

<67987x26293 sparse matrix of type '<class 'numpy.int64'>'
	with 436208 stored elements in Compressed Sparse Row format>

In [92]:
test_title = cvec.transform(X_test)
test_title
# test_title = np.matrix(test_title)

<16997x26293 sparse matrix of type '<class 'numpy.int64'>'
	with 104688 stored elements in Compressed Sparse Row format>

In [93]:
X_he = news_data.Headline.fillna('')
y_he = news_data.popularity

train_head,test_head,y_train,y_test = train_test_split(X_he , y_he, test_size = 0.20, shuffle= False)


In [94]:
train_headline = cvec.fit_transform(train_head)
train_headline



<67987x41138 sparse matrix of type '<class 'numpy.int64'>'
	with 1004292 stored elements in Compressed Sparse Row format>

In [95]:
test_headline = cvec.transform(test_head)

test_headline

<16997x41138 sparse matrix of type '<class 'numpy.int64'>'
	with 240065 stored elements in Compressed Sparse Row format>

In [96]:
# X_topic = news_data.Topic
# y_topic = news_data.popularity

# X_tr,X_te,y_train,y_test = train_test_split(X_topic , y_topic, test_size = 0.33)


In [97]:
# train_topic = cvec.fit_transform(X_tr)

# test_topic = cvec.transform(X_te)



In [98]:
# train_topic

In [99]:
# test_topic

In [100]:
import scipy.sparse as sp

train_title = sp.csr_matrix(train_title)
train_headline = sp.csr_matrix(train_headline)
# train_topic = sp.csr_matrix(train_topic)

train = sp.hstack((train_title, train_headline), format = 'csr')

train

<67987x67431 sparse matrix of type '<class 'numpy.int64'>'
	with 1440500 stored elements in Compressed Sparse Row format>

In [101]:

# test_headline_mat = np.matrix(test_headline)

In [102]:
import scipy.sparse as sp

test_title = sp.csr_matrix(test_title)
test_headline = sp.csr_matrix(test_headline)
# test_topic = sp.csr_matrix(test_topic)
test = sp.hstack((test_title, test_headline), format = 'csr')

test

<16997x67431 sparse matrix of type '<class 'numpy.int64'>'
	with 344753 stored elements in Compressed Sparse Row format>

In [145]:
lr = KNeighborsClassifier(n_neighbors = 10)
# lr = DecisionTreeClassifier(random_state = 0)
# lr = LogisticRegression()
# lr = SGDClassifier(loss = 'hinge', penalty = 'L2', alpha = 1e-3, random_state = 42, max_iter = 5, tol = None)
# lr = MultinomialNB()
# lr = DecisionTreeClassifier()

In [146]:
lr.fit(train, y_train)
# lr.predict(test)
start_time = time.time()
y_predict = lr.predict(test)

# acc_train = accuracy_score(y_train, lr.predict(train))
acc_test = accuracy_score(y_test, y_predict)
print(time.time()-start_time)

# print('Acc of Knearest on training :%2f' %acc_train)
print('Acc of Knearest on test :%2f' %acc_test)

77.83399367332458
Acc of Knearest on test :0.715244


In [112]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.62      0.05      0.10      2720
           1       0.85      0.99      0.91     14277

   micro avg       0.84      0.84      0.84     16997
   macro avg       0.73      0.52      0.51     16997
weighted avg       0.81      0.84      0.78     16997



In [113]:
conf_matrix = confusion_matrix(y_test,y_predict)

conf_matrix

array([[  147,  2573],
       [   91, 14186]], dtype=int64)

In [54]:
total2 = sum(sum(conf_matrix))
#####from confusion matrix calculate accuracy
accuracy2 = (conf_matrix[0,0]+conf_matrix[1,1])/total2
accuracy2

0.796140495381538

In [117]:
precis = (conf_matrix[1,1]/(conf_matrix[1,1]+conf_matrix[1,0]))

precis

0.9936261119282762